In [24]:
import tkinter as tk
from tkinter import filedialog
from datetime import datetime, timedelta
import os
from PIL import Image, ImageTk
import numpy as np
import random
import math
import re
import os
import xlsxwriter

import seaborn as sns
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
# def getType():

# #     global max_days

# #     if MaxDays.get() == "":
# #         max_days = "3,10"
# #     else:
# #         max_days = MaxDays.get()

def get_material():
    global df

    print('Decision Support System(DSS) :-)')
    import_file_path = filedialog.askopenfilename()
    print('Importing...')
    print('Please Wait :)')
    missing_values = ["n/a", "na", "--", "nan", ".", "-", "- ", "_"]
    df = pd.read_excel(import_file_path, na_values=missing_values)
    print('Decision Support System(DSS)')
    print('Files have been Imported\nClick Run and close the window!')



In [45]:
print('Please Import Excel Files')

root = tk.Tk()
root.geometry('550x450')
root.title("Decision Support System(DSS)")

label_0 = tk.Label(root, text="Heart Disease Recognition", relief="solid", width=30, font=("arial", 19, "bold"))
label_0.place(x=300, y=105, anchor="center")

# MaxDays = tk.StringVar()
# LMaxDays = tk.Label(root, text="Max Days: ", width=10, font=("bold", 10))
# LMaxDays.place(x=175, y=270, anchor="center")
# EMaxDays = tk.Entry(root, textvar=MaxDays, width=5)
# EMaxDays.place(x=235, y=270, anchor="center")

# max_days = None
df = None
# df_scheme= None
# df_ride= None

Get_List = tk.Button(root, text='Heart Disease Data', width=20, bg='brown', fg='white',
                     command=get_material).place(x=300, y=250, anchor="center")

Get_Max = tk.Button(root, text='Run', width=15, bg='green', fg='white').place(x=300, y=350, anchor="center")

root.mainloop()

# del Get_List
# del Get_Ride
# del Get_Max
# del Get_DailyFraud
# del Get_HistFraud

Please Import Excel Files
Decision Support System(DSS) :-)
Importing...
Please Wait :)
Decision Support System(DSS)
Files have been Imported
Click Run and close the window!


In [47]:
class1=data1[data1['Target']==1]
class2=data1[data1['Target']==2]
class3=data1[data1['Target']==3]
class4=data1[data1['Target']==4]
class5=data1[data1['Target']==5]

NameError: name 'data1' is not defined